In [1]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 


def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)



def matrix_fractional_differentiation_linear_plus(x,w,b,a):
    
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    '''
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    '''
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test


#ifogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        t0, t1 = t.tensor([0.]),t.tensor([1.])
        t0 = t0.to(device)
        t1 = t1.to(device)
        y1 = t.where(input1 - target <= 0, t0, t1)
        y_pred_f = t.abs(input1 - target)
        grad_output1 = gamma(3)*y_pred_f**(2-a)/gamma(3-a)*y1
        y2 = t.where(input1 - target > 0, t0, t1)
        grad_output2 = 2*(input1 - target)*y2

        grad_output = (grad_output1 + grad_output2)/input1.numel()
        
        return grad_output,None
    
#ifogd_Linear
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a)   #IFOGD
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    

In [4]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
time1 = time.time()
slide_windows_size = 36  #i.e.,input length
stock_array = ['GSPC','DJI','IXIC','FTSE','GDAXI','N225']   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
#stock_array = ['ETTh1','ETTh2','ETTm1','ETTm2']   #ETTh1，ETTh2，ETTm1，ETTm2
for stock in stock_array:
    #stock = 'DJI'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
    pre_days = 48     #i.e.,prediction lengths,48,96,192,384
    while pre_days <= 384:  
        df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
        del df_DJIA['Date']
        #del df_DJIA['date']        #ETT
        sca_DJIA = scaler.fit_transform(df_DJIA)
        label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)
        #label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
        sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
        #label
        deq_DJIA = deque(maxlen=pre_days)
        DJIA_label = []
        for i in label:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==pre_days:
                DJIA_label.append(list(deq_DJIA))
        DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
        #X
        deq_DJIA = deque(maxlen=slide_windows_size)
        DJIA_X = []
        for i in sca_DJIA:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==slide_windows_size:
                DJIA_X.append(list(deq_DJIA))
        DJIA_X = t.Tensor(DJIA_X)
        X_DJIA, y_DJIA = DJIA_X.to(device), DJIA_label.to(device)
        X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1

        class Net(nn.Module):
            def __init__(self,):
                super(Net,self).__init__()
                self.linear1 = Linear(slide_windows_size*5,128)
                #self.linear1 = Linear(slide_windows_size*7,128)  #ETT
                self.linear2 = Linear(128,64)
                self.linear3 = Linear(64,32)
                self.linear4 = Linear(32,pre_days)
                self.w1,self.b1 = self.linear1.weight,self.linear1.bias
                self.w2,self.b2 = self.linear2.weight,self.linear2.bias
                self.w3,self.b3 = self.linear3.weight,self.linear3.bias
                self.w4,self.b4 = self.linear4.weight,self.linear4.bias

            def forward(self,x):
                x = x.view(-1,slide_windows_size*5)
                #x = x.view(-1,slide_windows_size*7)   #ETT
                y2 = Linear_fractional.apply(x,self.w1,self.b1)   
                y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
                y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
                y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
                return y5

        net = Net()
        torch.save(net.state_dict(), r'./model/RMSprop_Stock/'+stock+'_'+str(pre_days)+'_net.pth')    

        a = 0.6
        lr =5e-5   #rmsprop:5e-5
        epoch = 1000
        batch_size = 256
        train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11 = [],[],[],[],[]  #collecting train set loss
        while a < 1.1:
            a = round(a + 0.1,1)
            best_loss = 10
            net = Net()
            net.load_state_dict(torch.load(r'./model/RMSprop_Stock/'+stock+'_'+str(pre_days)+'_net.pth'))
            optimizer = torch.optim.RMSprop(net.parameters(),lr=lr)
            net.to(device)
            for i in range(epoch):
                loss_sum = 0
                net.train()  
                start = np.random.randint(11,size=1)[0]  #Generate a random integer between 0 and 10
                while start+batch_size<len(X_train):
                    X_batch = X_train[start:start+batch_size]
                    y_batch = y_train[start:start+batch_size]
                    start = start+batch_size
                    out = net(X_batch)
                    loss = MSELoss_fractional.apply(out,y_batch) 
                    loss.backward()
                    loss_sum += loss
                    optimizer.step()
                    net.zero_grad()
                if a==0.7:
                    train_loss_07.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.8:
                    train_loss_08.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.9:
                    train_loss_09.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.0:
                    train_loss_10.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.1:
                    train_loss_11.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                net.eval()
                with torch.no_grad():
                    out = net(X_val)
                    MSE_val = MSE(y_val.cpu().detach().numpy(),out.cpu().detach().numpy())   #MSE，MAPE
                    if best_loss > MSE_val:
                        best_loss = MSE_val
                        torch.save(net.state_dict(), r'./model/RMSprop_Stock/'+stock+'_net_fractional_'+str(a).replace(".", "_")+'_'+str(pre_days)+'.pth')
                        print('epoch:{0}'.format(i+1)+'\n'+'Validation Loss:{0}'.format(MSE_val))
            print('the order'+str(a)+'finished!')
        if pre_days==48:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/RMSprop_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==96:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/RMSprop_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==192:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/RMSprop_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==384:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/RMSprop_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        pre_days = pre_days*2
time2 = time.time()
print(time2 - time1)

epoch:1
Validation Loss:1.0009937286376953
epoch:2
Validation Loss:0.9882777333259583
epoch:37
Validation Loss:0.8311494588851929
epoch:38
Validation Loss:0.6596436500549316
epoch:39
Validation Loss:0.5233726501464844
epoch:40
Validation Loss:0.4166402518749237
epoch:41
Validation Loss:0.3340025246143341
epoch:42
Validation Loss:0.27061912417411804
epoch:43
Validation Loss:0.2225864827632904
epoch:44
Validation Loss:0.18633481860160828
epoch:45
Validation Loss:0.1591683179140091
epoch:46
Validation Loss:0.13892969489097595
epoch:47
Validation Loss:0.12387170642614365
epoch:48
Validation Loss:0.11269568651914597
epoch:49
Validation Loss:0.1043546050786972
0.7521685
epoch:50
Validation Loss:0.09814160317182541
epoch:51
Validation Loss:0.09335407614707947
epoch:52
Validation Loss:0.09001149982213974
epoch:53
Validation Loss:0.08736200630664825
epoch:54
Validation Loss:0.08530137687921524
epoch:55
Validation Loss:0.08374850451946259
epoch:56
Validation Loss:0.08260724693536758
epoch:57
Val

0.7286659
0.16783997
epoch:342
Validation Loss:0.08114875853061676
epoch:343
Validation Loss:0.08034814149141312
epoch:344
Validation Loss:0.07908231765031815
epoch:345
Validation Loss:0.07879212498664856
epoch:346
Validation Loss:0.07829312980175018
epoch:347
Validation Loss:0.0772174745798111
epoch:348
Validation Loss:0.0771360695362091
epoch:349
Validation Loss:0.0768185630440712
0.14702475
epoch:350
Validation Loss:0.07651936262845993
epoch:351
Validation Loss:0.07614623755216599
epoch:352
Validation Loss:0.07590353488922119
epoch:353
Validation Loss:0.07534191012382507
epoch:354
Validation Loss:0.07519248872995377
epoch:357
Validation Loss:0.0749996230006218
epoch:359
Validation Loss:0.07472258806228638
epoch:360
Validation Loss:0.07469186931848526
epoch:363
Validation Loss:0.07459648698568344
epoch:365
Validation Loss:0.07422125339508057
epoch:366
Validation Loss:0.07375440746545792
epoch:370
Validation Loss:0.07096920162439346
epoch:372
Validation Loss:0.07091217488050461
epoch:

epoch:82
Validation Loss:0.08012208342552185
epoch:83
Validation Loss:0.07995273917913437
epoch:84
Validation Loss:0.0795886367559433
epoch:85
Validation Loss:0.07926274091005325
epoch:86
Validation Loss:0.07908938825130463
epoch:88
Validation Loss:0.07860007882118225
epoch:90
Validation Loss:0.07787539809942245
epoch:91
Validation Loss:0.07699091732501984
epoch:92
Validation Loss:0.07681415230035782
epoch:94
Validation Loss:0.07669256627559662
epoch:95
Validation Loss:0.07618371397256851
epoch:96
Validation Loss:0.07589440792798996
epoch:97
Validation Loss:0.07562181353569031
epoch:98
Validation Loss:0.07523813098669052
epoch:99
Validation Loss:0.07490608096122742
0.15644617
epoch:100
Validation Loss:0.07470393925905228
epoch:101
Validation Loss:0.07398524135351181
epoch:102
Validation Loss:0.07366922497749329
epoch:103
Validation Loss:0.07353668659925461
epoch:104
Validation Loss:0.07327292114496231
epoch:105
Validation Loss:0.07207474857568741
epoch:108
Validation Loss:0.07179528474

epoch:4
Validation Loss:0.3178010582923889
epoch:5
Validation Loss:0.24731798470020294
epoch:6
Validation Loss:0.19452393054962158
epoch:7
Validation Loss:0.15595275163650513
epoch:8
Validation Loss:0.12846794724464417
epoch:9
Validation Loss:0.10939402878284454
epoch:10
Validation Loss:0.09638853371143341
epoch:11
Validation Loss:0.08757069706916809
epoch:12
Validation Loss:0.0816701203584671
epoch:13
Validation Loss:0.07771609723567963
epoch:14
Validation Loss:0.07508470118045807
epoch:15
Validation Loss:0.07330108433961868
epoch:16
Validation Loss:0.07203255593776703
epoch:17
Validation Loss:0.07107871025800705
epoch:18
Validation Loss:0.07038461416959763
epoch:19
Validation Loss:0.06987236440181732
epoch:20
Validation Loss:0.0693071261048317
epoch:21
Validation Loss:0.06886346638202667
epoch:22
Validation Loss:0.06845340877771378
epoch:23
Validation Loss:0.06802493333816528
epoch:24
Validation Loss:0.06756742298603058
epoch:25
Validation Loss:0.06718573719263077
epoch:26
Validation

epoch:184
Validation Loss:0.032802775502204895
epoch:185
Validation Loss:0.03273653984069824
epoch:186
Validation Loss:0.0326666533946991
epoch:187
Validation Loss:0.03259634971618652
epoch:188
Validation Loss:0.0325339175760746
epoch:189
Validation Loss:0.03247274458408356
epoch:190
Validation Loss:0.03240548074245453
epoch:191
Validation Loss:0.03234255313873291
epoch:192
Validation Loss:0.032280948013067245
epoch:193
Validation Loss:0.032212838530540466
epoch:194
Validation Loss:0.032151855528354645
epoch:195
Validation Loss:0.03210050240159035
epoch:196
Validation Loss:0.03205113112926483
epoch:197
Validation Loss:0.03198975697159767
epoch:198
Validation Loss:0.03192872554063797
epoch:199
Validation Loss:0.031865570694208145
0.08207931
epoch:200
Validation Loss:0.03181049972772598
epoch:201
Validation Loss:0.031762123107910156
epoch:202
Validation Loss:0.031712617725133896
epoch:203
Validation Loss:0.03165653347969055
epoch:204
Validation Loss:0.03160213306546211
epoch:205
Validati

...

epoch:24
Validation Loss:0.7053585648536682
epoch:25
Validation Loss:0.6868317723274231
epoch:26
Validation Loss:0.6719709634780884
epoch:27
Validation Loss:0.6601500511169434
epoch:28
Validation Loss:0.647779643535614
epoch:29
Validation Loss:0.6340234875679016
epoch:30
Validation Loss:0.6210492253303528
epoch:31
Validation Loss:0.6075164675712585
epoch:32
Validation Loss:0.5908511281013489
epoch:33
Validation Loss:0.5765495896339417
epoch:34
Validation Loss:0.5597772598266602
epoch:35
Validation Loss:0.5506643652915955
epoch:36
Validation Loss:0.538589358329773
epoch:37
Validation Loss:0.5249640345573425
epoch:39
Validation Loss:0.5118368268013
epoch:40
Validation Loss:0.5003519058227539
epoch:41
Validation Loss:0.48659422993659973
epoch:43
Validation Loss:0.4819490313529968
epoch:44
Validation Loss:0.4681123197078705
epoch:45
Validation Loss:0.4571913778781891
epoch:46
Validation Loss:0.44550833106040955
epoch:47
Validation Loss:0.43522971868515015
epoch:48
Validation Loss:0.4296050

1.1081378
1.0947005
1.0939659
1.0887867
1.0892099
1.0907031
1.0973163
1.0993006
1.1023936
1.108742
1.1060308
1.1081167
1.1112092
1.1127743
1.1139717
1.1152154
1.1199893
1.1200721
1.1201401
the order0.7finished!
epoch:1
Validation Loss:0.9260819554328918
epoch:2
Validation Loss:0.9087438583374023
epoch:3
Validation Loss:0.8880789279937744
epoch:4
Validation Loss:0.8646914958953857
epoch:5
Validation Loss:0.8367053866386414
epoch:6
Validation Loss:0.8041056394577026
epoch:7
Validation Loss:0.7662897706031799
epoch:8
Validation Loss:0.7247713208198547
epoch:9
Validation Loss:0.6848723292350769
epoch:10
Validation Loss:0.6504285931587219
epoch:11
Validation Loss:0.6110258102416992
epoch:12
Validation Loss:0.5725590586662292
epoch:13
Validation Loss:0.5378903150558472
epoch:14
Validation Loss:0.5061281323432922
epoch:15
Validation Loss:0.4734855890274048
epoch:16
Validation Loss:0.44479238986968994
epoch:17
Validation Loss:0.41888195276260376
epoch:18
Validation Loss:0.3965194821357727
epoc

epoch:120
Validation Loss:0.21969041228294373
1.0788001
1.0648551
1.0577408
1.0568863
1.0557511
1.0594611
1.0610619
1.0635972
1.0708749
1.0800374
1.0864108
1.0959319
1.1076536
1.1128554
1.1257812
1.1321789
1.1468326
1.156199
the order0.9finished!
epoch:1
Validation Loss:0.8439366221427917
epoch:2
Validation Loss:0.7453373074531555
epoch:3
Validation Loss:0.656426191329956
epoch:4
Validation Loss:0.5831215977668762
epoch:5
Validation Loss:0.5241491794586182
epoch:6
Validation Loss:0.4763873517513275
epoch:7
Validation Loss:0.43715572357177734
epoch:8
Validation Loss:0.40482965111732483
epoch:9
Validation Loss:0.37773391604423523
epoch:10
Validation Loss:0.3549654483795166
epoch:11
Validation Loss:0.3353760838508606
epoch:12
Validation Loss:0.31871190667152405
epoch:13
Validation Loss:0.3044232726097107
epoch:14
Validation Loss:0.29216650128364563
epoch:15
Validation Loss:0.28146201372146606
epoch:16
Validation Loss:0.27234992384910583
epoch:17
Validation Loss:0.26453855633735657
epoch:1

epoch:31
Validation Loss:0.30388912558555603
epoch:32
Validation Loss:0.30111023783683777
epoch:33
Validation Loss:0.2988397479057312
epoch:34
Validation Loss:0.2967458963394165
epoch:35
Validation Loss:0.2949856221675873
epoch:36
Validation Loss:0.29346442222595215
epoch:37
Validation Loss:0.29250767827033997
epoch:38
Validation Loss:0.29124927520751953
epoch:39
Validation Loss:0.290274441242218
epoch:40
Validation Loss:0.28947821259498596
epoch:41
Validation Loss:0.2890087068080902
epoch:42
Validation Loss:0.28831344842910767
epoch:43
Validation Loss:0.28786274790763855
epoch:44
Validation Loss:0.28747817873954773
epoch:45
Validation Loss:0.2871572971343994
epoch:46
Validation Loss:0.2869194746017456
epoch:47
Validation Loss:0.28666043281555176
epoch:48
Validation Loss:0.2864746153354645
epoch:49
Validation Loss:0.28626567125320435
1.9455756
epoch:50
Validation Loss:0.28605031967163086
epoch:52
Validation Loss:0.28596532344818115
epoch:53
Validation Loss:0.28588563203811646
epoch:54


epoch:26
Validation Loss:0.2850244343280792
epoch:27
Validation Loss:0.2846870422363281
epoch:28
Validation Loss:0.2844090461730957
epoch:29
Validation Loss:0.28428372740745544
epoch:30
Validation Loss:0.28416407108306885
epoch:31
Validation Loss:0.28404995799064636
epoch:32
Validation Loss:0.2840476334095001
1.9061866
1.88788
1.8714286
1.8566788
1.8434259
1.830907
1.8167529
1.8059089
1.799181
1.7908587
1.784285
1.7818818
1.7777387
1.7776514
1.7714837
1.7730894
1.7737727
1.7725707
1.7670172
1.7704445
the order1.0finished!
epoch:1
Validation Loss:0.8172492384910583
epoch:2
Validation Loss:0.8092813491821289
epoch:3
Validation Loss:0.8038320541381836
epoch:4
Validation Loss:0.8011233806610107
epoch:5
Validation Loss:0.7958406209945679
epoch:6
Validation Loss:0.78975510597229
epoch:7
Validation Loss:0.7817266583442688
epoch:8
Validation Loss:0.7728111147880554
epoch:9
Validation Loss:0.7654107809066772
epoch:10
Validation Loss:0.7526317238807678
epoch:11
Validation Loss:0.7397856116294861